## Lecture 16 EEP118

In [ ]:
#Lecture16.R
#LECTURE 16



# Load the 'pacman' package
library(pacman) 
#packages to use load them now using the pacman "manager"
p_load(dplyr, haven, readr) 
#Another great feature of p_load(): if you try to load a package that is not installed on your machine, p_load() install the package for you, rather than throwing an error. For instance, let’s install and load one final package named ggplot2. 
p_load(ggplot2)

pacman::p_load(lfe, lmtest, haven, sandwich, tidyverse)
# lfe for running fixed effects regression
# lmtest for displaying robust SE in output table
# haven for loading in dta files
# sandwich for producing robust Var-Cov matrix
# tidyverse for manipulating data and producing plots

In [ ]:

#set scientific display off, thank you Roy
options(scipen=999)

#read in a Stata dataset
mydata <- read_dta("Lecture16.dta")
head(mydata)

In [ ]:
#generate category education variable
mydata$cateduc=1
mydata$cateduc[mydata$educ==12]<-2
mydata$cateduc[mydata$educ>12]<-3

#summary stats variables
summary(mydata)

### *1. ESTIMATE DIFFERENCE IN MEANS BETWEEN MALE AND FEMALE

In [ ]:
#*1. ESTIMATE DIFFERENCE IN MEANS BETWEEN MALE AND FEMALE

summary(mydata$wage[which(mydata$female==1)])
summary(mydata$wage[which(mydata$female==0)])

#standard errors of the data
sdfemale<-sd(mydata$wage[which(mydata$female==1)])
sdmale<-sd(mydata$wage[which(mydata$female==0)])
sdfemale
sdmale

#number of observations
nfemale<-n_perc(mydata$female==1)
nmale<-n_perc(mydata$female==0)
nfemale
nmale

In [ ]:
# t test difference in means

t.test(mydata$wage~mydata$female)


graph of wage by female indicator

In [ ]:
ggplot(mydata, aes(x = factor(female), y = wage))+ 
  stat_summary(fun.y="mean", geom="bar")

### *2. DISTRIBUTION OF WAGE BY GENDER


In [ ]:

ggplot(mydata,aes(x=wage)) + 
  geom_histogram(data=subset(mydata,female == 1),fill = "red", alpha = 0.2) +
  geom_histogram(data=subset(mydata,female == 0),fill = "blue", alpha = 0.2)



In [ ]:
#linear model
two<-lm(wage~female,mydata)
summary(two)

### 3. DO WOMEN HAVE DIFFERENT CHARACTERISTICS THAT MATTER FOR WAGE?

In [ ]:


###graph average wages for different educaiton levels

mydata$wage_female=mydata$wage
mydata$wage_female[mydata$female==0]<-0

mydata$wage_male=mydata$wage
mydata$wage_male[mydata$female==1]<-0

mydata <- mydata %>% 
  group_by(educ,female) %>% 
  mutate(meanwage = mean(wage))


ggplot(mydata,aes(y = meanwage,x = educ,color = as.factor(female))) + 
  geom_line() 



#### #Discrimination, even after controlling for difference in characteristics:  

In [ ]:
#Discrimination, even after controlling for difference in characteristics:  
#Additive female effect

three<-lm(wage~female+educ+exper+services+profocc, mydata)
summary(three)

four<-lm(wage~female, mydata)
summary(four)


### *4. ARE THERE DIFFERENTIAL RETURNS TO EDUCATION FOR MEN AND WOMEN?

In [ ]:

#generate interaction
mydata$femeduc=mydata$female*mydata$educ

five<-lm(wage~female+educ+femeduc, mydata)
summary(five)


In [ ]:
#generate predictions
mydata$wagehat<-predict(lm(wage~female+educ+femeduc, mydata))

### plot them the wages for female and non female and also the predictions

In [ ]:


#mydata$trfem=mydata$wagehat
#mydata$trfem[mydata$female==0]<-0

#mydata$trmale=mydata$wagehat
#mydata$trmale[mydata$female==1]<-0

ggplot(mydata, aes(x=educ, y=wage, color=as.factor(female))) +
  geom_point() + 
  geom_smooth(method=lm, se=FALSE, fullrange=TRUE)



### #* Finally, is there a WEST COAST DIFFERENTIAL EFFECT ON MEN' AND WOMEN'S WAGES?

In [ ]:
#* Finally, is there a WEST COAST DIFFERENTIAL EFFECT ON MEN' AND WOMEN'S WAGES?
summary(mydata$west)

mydata$femwest=mydata$female*mydata$west

five<-lm(wage~female+west+femwest+educ+exper,mydata)
summary(five)